# Datawarehouse Single Column Insert.

#### With great power comes great responsibility, try not to nuke the data warehouse.

We operate downstream, and new columns are added to the ERP on a whim. 

Backfilling core reporting tables in their entirety takes an annoying amount of time, which is why this exists.

- Check data types before you import.
  - Use the query below, don't run all cells.
  - Do **NOT** import any columns with CLOB data types.
  - They are over 8,000 characters; datawarehouse doesn't like that.
- Make sure you check that id is a unique key.

In [ ]:
import pandas as pd
import pyodbc
import numpy as np
import os

from config import *

In [ ]:
# New column to be added.
new_column = ""
# The unique key you're joining the record by; be sure to check if it's actually unique.
key = ""
# NetSuite table name.
target_table = ""

In [ ]:
# Create a connection using pyodbc for NetSuite.
connection_string = f'DSN={dsn};UID={netsuite_user};PWD={netsuite_pw}'

# Execute the query; explicitly closing connections is cringe.
with pyodbc.connect(connection_string) as connection:
    with connection.cursor() as cursor:
        
        # Query; add columns from table needed here.
        query = f""" SELECT TABLE_NAME, COLUMN_NAME, REMARKS, DATA_TYPE, TYPE_NAME FROM OA_COLUMNS WHERE TABLE_NAME = '{target_table}' AND COLUMN_NAME = '{new_column}' """
        cursor.execute(query)
        
        # Fetch, define column names.
        columns = [column[0] for column in cursor.description]
        results = cursor.fetchall()

        # Convert to df.
        df = pd.DataFrame.from_records(results, columns=columns)

df

In [ ]:
# Create a connection using pyodbc for NetSuite.
connection_string = f'DSN={dsn};UID={netsuite_user};PWD={netsuite_pw}'

# Execute the query; explicitly closing connections is cringe.
with pyodbc.connect(connection_string) as connection:
    with connection.cursor() as cursor:
        
        # Query; add columns from table needed here.
        query = 'SELECT '+key+', '+new_column+' FROM '+target_table+' WHERE '+new_column+' IS NOT NULL'
        cursor.execute(query)
        
        # Fetch, define column names.
        columns = [column[0] for column in cursor.description]
        results = cursor.fetchall()

        # Convert to df.
        df = pd.DataFrame.from_records(results, columns=columns)

df.shape

In [ ]:
# Create connection using pyodbc for local servers.
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER="+server+";DATABASE="+db+";UID="+local_user+";PWD="+local_pw+";Trusted_Connection=YES"

# We are creating a table to dump output from NetSuite into.
with pyodbc.connect(connection_string) as connection:
    with connection.cursor() as cursor:
        # Drop table if it exists
        cursor.execute("IF OBJECT_ID('Data_Imports.dbo.Column_Insert', 'U') IS NOT NULL DROP TABLE Data_Imports.dbo.Column_Insert")
        connection.commit()

        # Create a new table based on DataFrame's columns
        create_table_statement = "CREATE TABLE Data_Imports.dbo.Column_Insert ("
        create_table_statement += ", ".join([f"[{col}] VARCHAR(250)" for col in df.columns]) + ")"
        cursor.execute(create_table_statement)
        connection.commit()

        # Insert DataFrame into the new table
        for index, row in df.iterrows():
            placeholders = ', '.join('?' for _ in row)
            sql = "INSERT INTO Data_Imports.dbo.Column_Insert VALUES (" + placeholders + ")"
            cursor.execute(sql, tuple(row))
        
        connection.commit()

In [ ]:
# Check if column already exists in local server's table. If so, drop and create.
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER="+server+";DATABASE="+db2+";UID="+local_user+";PWD="+local_pw+";Trusted_Connection=YES"

with pyodbc.connect(connection_string) as connection:
    with connection.cursor() as cursor:
        # Check and drop column if it exists
        cursor.execute(f"""
        IF EXISTS (SELECT * FROM INFORMATION_SCHEMA.COLUMNS 
                   WHERE TABLE_SCHEMA = 'ns' 
                   AND TABLE_NAME = ? 
                   AND COLUMN_NAME = ?)
        BEGIN
            EXEC('ALTER TABLE NetSuite.ns.[' + ? + '] DROP COLUMN ' + ?);
        END
        """, (target_table, new_column, target_table, new_column))
        
        # Add new column
        cursor.execute(f"ALTER TABLE NetSuite.ns.[{target_table}] ADD {new_column} VARCHAR(250);")
        connection.commit()

In [ ]:
with pyodbc.connect(connection_string) as connection:
    with connection.cursor() as cursor:
        # Execute the UPDATE statement with JOIN
        sql_update = f"""
        UPDATE A
        SET A.{new_column} = B.{new_column}
        FROM NetSuite.ns.[{target_table}] A
        INNER JOIN DATA_IMPORTS.dbo.Column_Insert B ON A.{key} = B.{key};
        """
        cursor.execute(sql_update)
        connection.commit()